In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("zameen-updated.csv")
print(df.shape)
df.head()

In [ ]:
df.describe()

# Counting Null Values

In [ ]:
# Null values in each column
null_values = df.isna().sum()
null_values

In [ ]:
# Missing values in each column
missing_values = df.isnull().sum()
missing_values

In [ ]:
#drop url column as not much need 
df = df.drop(["page_url","property_id"], axis = 1)

# Creating New Columns

In [ ]:
#filling vacant space to 0
df.fillna(0)

# To Standardize area. Converting kanal to marla
kanal_to_marla = 20

def conversion(row):
    if row['Area Type'] == "Kanal":
        return row['Area Size'] * kanal_to_marla 
    elif row['Area Type'] == "Marla":
        return row['Area Size']

df['Area(Marla)'] = df.apply(conversion, axis=1)

#We are also extracting Date, Month, Year from Date to new columns (might needed in future)

df['date_added'] = pd.to_datetime(df["date_added"], format="%m/%d/%Y")

df.insert(14, "Day", df["date_added"].dt.day)
df.insert(15, "Month", df["date_added"].dt.month)
df.insert(16, "Year", df["date_added"].dt.year) #dt is accessor

df.head()

# Cleaning Data

In [ ]:
#Dropping Agency and agent column
df2 = df.drop(["agent","agency"], axis=1)

### By BedRoom and Baths

In [ ]:
df2[(df2['baths']==0) | (df2['bedrooms']==0)]

In [ ]:
#droping entries having no bed and baths
df2 = df2.drop(df2[(df2['baths']==0) | (df2['bedrooms']==0)].index)

In [ ]:
df2[df2['bedrooms']>13]

In [ ]:
df2 = df2.drop(df2[df2['bedrooms'] > 13].index)
print(df2.shape)
df2['baths'].unique()

In [ ]:
#We will also remove those having rooms > 3 and baths == 0, quite unusual
df2 = df2.drop(df2[(df2['baths']==0) & (df2['bedrooms'] > 3)].index)

In [ ]:
#Also dropping for those having baths > (rooms+3)
df2[(df2['baths']) > (df2['bedrooms']+3)]

In [ ]:
df2 = df2.drop(df2[df2['baths']>(df2['bedrooms']+3)].index)
df2.shape

### Outliers 

In [ ]:
#No. of Non-duplicates (location key)
values_after_deduplication = df['location_id'].nunique() 
print(values_after_deduplication)

Droping entries having price < 3000 For Rent and price <100000 For Sale

In [ ]:
df2 = df2.drop(df2[df2['price']<3000].index)

In [ ]:
pd.set_option('display.max_columns', None)
df2 = df2.drop(df2[(df2['price']<100000) & (df2['purpose']=='For Sale')].index) 

In [ ]:
#Removing those having Area(marla) == 0
df2 = df2.drop(df2[df2['Area(Marla)']==0].index)

We will create another column 'price per marla', so that we can compare it with HomeTown avg. price, to remove outliers

In [ ]:
df2['Price per Marla'] = df2['price'] / df2['Area(Marla)']

In [ ]:
#Droping price (marla) > 6,000,000
df2 = df2.drop(df2[df2['Price per Marla']>6000000].index)

In [ ]:
len(df['location'].unique())

In [ ]:
#Removing any leading or trailing whitespaces from each location entry
df['location'] = df['location'].apply(lambda x: x.strip()) 

#Count Occurence of each location value
location_stats = df2['location'].value_counts(ascending=False)

location_stats.head(40)

But it is not mention which Bahria Town, Cantt or DHA Defence it is.
We will now concat city with each of them

In [ ]:
locations = ["Bahria Town", "Askari", "DHA Defence","Gulberg"]
df2['location'] = np.where((df2['location'].isin(locations)), 
                              df2['location'] + ' ' + df2['city'],
                              df2['location'])

In [ ]:
#Count Occurence of each location value Once Again
location_stats = df2['location'].value_counts(ascending=False)

location_stats.head(40)

Crating 2 Seperate dataFrame For Sale and For Rent

In [ ]:
df_sale = df2[df2["purpose"]== "For Sale"]
df_rent = df2[df2["purpose"]== "For Rent"]
df_rent

In [ ]:
def identify_outliers(group):
    #Droping value, Lower Bound = 10th Percentile & Upper Bound = 90th Perc
    Q1 = group['Price per Marla'].quantile(0.10)
    Q3 = group['Price per Marla'].quantile(0.90)
    IQR = Q3 - Q1
    return group[(group['Price per Marla'] >= Q1 - 1.5 * IQR) & 
                 (group['Price per Marla'] <= Q3 + 1.5 * IQR)]


# Apply the filter_outliers function to each group of locations
df_sale = df_sale.groupby('location').apply(identify_outliers)
df_rent = df_rent.groupby('location').apply(identify_outliers)

# Reset the index of the resulting DataFrame
df_sale.reset_index(drop=True, inplace=True)
df_rent.reset_index(drop=True, inplace=True)

df_rent

### Droping Duplicates
There are unusual duplicate values in this Dataset.
We will drop these with location_key,

In [ ]:
#No. of Non-duplicates (location key)
values_after_deduplication = df_rent['location_id'].nunique() 
print("df_Rent:", values_after_deduplication)

values_after_deduplication = df_sale['location_id'].nunique() 
print("df_sale:", values_after_deduplication)

We will retain only 1 value for each location, although there might be more than 1 entries for same location (As there are many portions in Flats)

In [ ]:
#TO KEEP ONLY LATEST ENTRY FOR A LOCATION
#Sort it by date and pick up the latest one

df_sale['date_added'] = pd.to_datetime(df_sale['date_added']) 
df_sale = df_sale.sort_values(by=['location_id', 'date_added'], ascending=[True, False])
df_sale = df_sale.drop_duplicates(subset='location_id', keep='first')
df_sale

In [ ]:
df_rent['date_added'] = pd.to_datetime(df_rent['date_added']) 
df_rent = df_rent.sort_values(by=['location_id', 'date_added'], ascending=[True, False])
df_rent = df_rent.drop_duplicates(subset='location_id', keep='first')
df_rent

In [ ]:
#Count Occurence of each location value in Sale df
location_stats_sale = df_sale['location'].value_counts(ascending=False)

location_stats_sale.head(40)

In [ ]:
#Count Occurence of each location value in rent df
location_stats_rent = df_rent['location'].value_counts(ascending=False)

location_stats_rent.head(40)

# Data Visualize

In [ ]:
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.express as px
%matplotlib inline

## Street Map

In [ ]:
min_value = df_sale['Price per Marla'].min()
max_value = df_sale['Price per Marla'].max()

fig = px.scatter_mapbox(
    df_sale,
    lat="latitude",
    lon="longitude",
    hover_name="location_id",
    hover_data = ["property_type","price","location"],
    color="Price per Marla",
    range_color=(min_value, max_value),
    zoom=10
)

fig.update_layout(mapbox_style="open-street-map")
pio.show(fig)

## Rooms vs Price Per Marla Plot (of Specific Location)

In [ ]:
def scatter_chart(df, location):
    One_Four_Beds = df[(df['location'] == location) & (df['bedrooms'] >= 1) & (df['bedrooms'] <= 4)]
    Four_plus_Beds = df[(df['location'] == location) & (df['bedrooms'] > 4)]

    plt.scatter(One_Four_Beds['Area(Marla)'], One_Four_Beds['price']/100000, color='blue', label="1 to 4 Bedrooms", s=50)
    plt.scatter(Four_plus_Beds ['Area(Marla)'], Four_plus_Beds['price']/100000, marker='+', color="green", label="4+ Bedrooms", s=50)
    plt.rcParams["figure.figsize"] = (15,10)
    plt.xlabel("Size per Marla")
    plt.ylabel("Price (Lac)")
    plt.title(location)
    plt.legend()

### For Sale House

In [ ]:
scatter_chart(df_sale, "Gulshan-e-Iqbal Town")

In [ ]:
scatter_chart(df_sale, "Malir")

### For Rent

In [ ]:
scatter_chart(df_rent, "E-11")

In [ ]:
scatter_chart(df_rent, "North Karachi")

## Plotting Most Expensive Locations in each City

In [ ]:
def expensive_locations(dataset, city_name):
    # Dataset for the specified city
    df_city = dataset[dataset['city'] == city_name]

    counts = df_city['location'].value_counts()

    # Only locations with more than 5 entries
    valid_locations = counts[counts > 5].index
    df_city = df_city[df_city['location'].isin(valid_locations)]

    # Calculate avg price per Marla
    df_expensive = df_city.groupby('location')['price'].mean().reset_index()

    # Sorting locations by avg price in descending order
    df_expensive = df_expensive.sort_values(by='price', ascending=False)

    # Selecting only the top most expensive locations
    df_top_locations = df_expensive.head(15)

    plt.bar(df_top_locations['location'], df_top_locations['price'] / 100000, color='skyblue')
    plt.xlabel('Location')
    plt.ylabel('Average Price per Marla (Lac)')
    plt.title(f' in {city_name}', fontsize=15)
    plt.xticks(rotation=45, ha='right')

### For Sale

In [ ]:
plt.figure(figsize=(10, 12))
plt.suptitle("For Sale", fontsize = 20, y=1)

plt.subplot(3, 1, 1)
expensive_locations(df_sale, 'Karachi')

plt.subplot(3, 1, 2)
expensive_locations(df_sale, 'Lahore')

plt.subplot(3, 1, 3)
expensive_locations(df_sale, 'Islamabad')

plt.tight_layout()
plt.show()

### For Rent

In [ ]:
plt.figure(figsize=(15, 12))
plt.suptitle("For Rent", fontsize = 25, y=1, horizontalalignment='center')
plt.subplot(2, 1, 1)
expensive_locations(df_rent, 'Karachi')

plt.subplot(2, 1, 2)
expensive_locations(df_rent, 'Islamabad')

plt.tight_layout()
plt.show()

In [ ]:
df_sale

## Plotting Different Property Types

In [ ]:
df_sale['property_type'].unique()

In [ ]:
#df2[df2['property_type']=='Farm House']

### For Sale

In [ ]:
df_sale_propTypes = df_sale['property_type'].value_counts()
pd.DataFrame(df_sale_propTypes)

## For Rent

In [ ]:
df_rent_propTypes = df_rent['property_type'].value_counts()
pd.DataFrame(df_rent_propTypes)